In [23]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict

model_rdf = joblib.load('./data/model_rdf.plk')
model_svm = joblib.load('./data/model_svm.plk')

rp_encoder = LabelEncoder()
rp_encoder.classes_ = np.load('./data/classes.npy', allow_pickle=True)

bssid_encoder = LabelEncoder()
bssid_encoder.classes_ = np.load('./data/features.npy', allow_pickle=True)

In [28]:
# detection용 유저 데이터를 잠시 학습용 데이터에서 가져옴.
# 실제 유저 데이터를 가져오면 bssid를 rp_encoder에서 transform 해야됨
# 데이터는 bssid(encoded)) : rssi 쌍의 dictionary -> dataframe으로 변환
df_signal = pd.read_csv(Path('./data/train_data.csv'), header=None)
dict_user = df_signal.sample().to_dict('records')[0]

# 유저 데이터 중 모델에 없는 bssid를 제외한 detection용 데이터
# 실제 데이터 쓸때는 bssid_num이 아닌 string bssid를 사용
dict_detection = defaultdict.fromkeys(bssid_encoder.transform(bssid_encoder.classes_), 0)
for bssid_num, rssi in dict_user.items():
    if bssid_num in dict_detection:
        dict_detection[bssid_num] = rssi

# string인 bssid를 숫자로 mapping
df_detection = pd.DataFrame().from_dict([dict_detection])

result_rdf = model_rdf.predict(df_detection)
result_svm = model_svm.predict(df_detection)
proba_rdf = model_rdf.predict_proba(df_detection)
proba_svm = model_svm.predict_proba(df_detection)
print(proba_rdf)

print(f'RDF : {rp_encoder.inverse_transform([result_rdf[0]])[0]}')
for rp in rp_encoder.classes_:
    print(f'[{rp}] : {proba_rdf[0][rp_encoder.transform([rp])[0]]:.2f}')

print('')

print(f'SVM : {rp_encoder.inverse_transform([result_svm[0]])[0]}')
for rp in rp_encoder.classes_:
    print(f'[{rp:7}] : {proba_rdf[0][rp_encoder.transform([rp])[0]]:.2f}')
    
print('\n')


RDF : ZARA
[2층 입구] : 0.00
[LLOYD] : 0.00
[ZARA] : 0.96
[ZIOZIA] : 0.03
[버스정류장] : 0.00
[서점] : 0.01

SVM : ZARA
[2층 입구] : 0.00
[LLOYD] : 0.00
[ZARA] : 0.96
[ZIOZIA] : 0.03
[버스정류장] : 0.00
[서점] : 0.01


